# Evaluate Baselines (Airfoil use case)

The goal of this notebook is to demonstrate how we can evaluate the results of a baseline on a given benchmark.

We will show how to load a baseline (or any other `AugmentedSimulator`) and evaluate it on a `Benchmark` of our choice.

**To learn more about the training procedure, visit [this notebook](https://github.com/IRT-SystemX/LIPS/blob/main/getting_started/PowerGridUsecase/03_TrainAnAugmentedSimulator.ipynb)**

### Prerequisites

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS) 

In [ ]:
# !pip install -r requirements.txt
# or 
# !pip install -U .

Install the AirfRANS package

In [ ]:
# !pip install airfrans

#### Import required packages

In [ ]:
import os
from lips import get_root_path
from lips.dataset.airfransDataSet import download_data
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark

In [ ]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [ ]:
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join("airfoilConfigurations","simulators","torch_fc.ini") #Configuration file re

## Initial step: download the data

In [ ]:
import os
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

#  Benchmark <a id="Case1"></a>

## First step: load the dataset

A common dataset will be used for evaluate the augmented simulator. This initial step aims at loading it once and for all.

In [ ]:
benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path = LOG_PATH)
benchmark.load(path=DIRECTORY_NAME)

Check the config is loaded appropriately for this benchmark

In [ ]:
print("Benchmark name: ", benchmark.config.section_name)

Each simulation is given as a point cloud defined via the nodes of the simulation mesh. Each point of a point cloud is described via 7 features:
- its position (two component in meter): 'x-position' and 'y-position'
- the inlet velocity (two components in meter per second): 'x-inlet_velocity' and 'y-inlet_velocity'
- the distance to the airfoil (one component in meter): distance_function
- the normals (two components in meter, set to 0 if the point is not on the airfoil): 'x-normals', 'y-normals'

In [ ]:
print("Input attributes (features): ", benchmark.config.get_option("attr_x"))

Each point is given a target of 4 components for the underlying regression task:
- the velocity (two components in meter per second): 'x-velocity'and 'y-velocity'
- the pressure divided by the specific mass (one component in meter squared per second squared): 'pressure'
- the turbulent kinematic viscosity (one component in meter squared per second): 'turbulent_viscosity'

In [ ]:
print("Output attributes (targets): ", benchmark.config.get_option("attr_y"))

The evaluation criteria can be found in the configure file

In [ ]:
print("Evaluation criteria: ")
criteria = benchmark.config.get_option("eval_dict")
for criteriaCategory,criteriaNames in criteria.items():
    print("\t Category %s: " %criteriaCategory," ",criteriaNames)

While the machine learning metrics are relatively standard, the physical metrics are closery related to the underlying use case and physical problem. We provide a description of each of them. 

There are two physical quantities considered
- the drag coefficient
- the lift coefficient

For each of them and for each dataset, we compute between the observations and predictions coefficients:
- the spearman correlation, a nonparametric measure of the monotonicity of the relationship between two datasets: 'spearman_correlation_drag', 'spearman_correlation_lift'
- the mean relative error: 'mean_relative_drag', 'mean_relative_lift'
- the standard deviation: 'std_relative_drag', 'std_relative_lift'

We can also have a look at the datasets (namely features, label, size of each variable)

In [ ]:
print("train dataset: ", benchmark.train_dataset)
print("test dataset: ", benchmark._test_dataset )
print("test dataset: ", benchmark._test_ood_dataset )

## A baseline "augmented simulator" <a id="bench1-fc"></a>

Along with some dataset, we provide also some baseline (from a trained neural network). This baseline is made of a fully connected neural network that takes the available input of the airfrans case and tries to predict all the output of the simulator.

The fully connected neural network is made of XXX layer each with YYY units.

It is learned for KKK epochs on the training set of the `Case1`.

First we need to load the baseline and initialize it properly

In [ ]:
from lips.augmented_simulators.torch_models.fully_connected import TorchFullyConnected
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative


chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
no_norm_x=benchmark.train_dataset.get_no_normalization_axis_indices()
scalerParams={"chunk_sizes":chunk_sizes,"no_norm_x":no_norm_x}

augmented_simulator = TorchSimulator(name="torch_fc",
                                     model=TorchFullyConnected,
                                     scaler=StandardScalerIterative,
                                     log_path="log_benchmark",
                                     device="cuda:0",
                                     bench_config_path=BENCH_CONFIG_PATH,
                                     bench_config_name=BENCHMARK_NAME,
                                     sim_config_path=SIM_CONFIG_PATH,
                                     sim_config_name="DEFAULT",
                                     architecture_type="Classical",
                                     scalerParams=scalerParams
                                    )

Training the neural network

In [ ]:
augmented_simulator.train(train_dataset=benchmark.train_dataset, epochs=2, train_batch_size=128000)

Then we can evaluate it on the test datasets of the benchmark. This is done by indicating the learned augmented simulator `augmented_simulator` as the argument of the `evaluate_simulator` method.

There are 2 steps performed within the evaluation of a simulator:
- Compute the prediction using the augmented simulator
- Evaluate the performances of the datasets within the benchmark with respect to the observation based on the prescribed metrics

In [ ]:
import time

start_test = time.time()
fc_metrics_test = benchmark.evaluate_simulator(dataset="test",augmented_simulator=augmented_simulator,eval_batch_size=256000 )
test_evaluation_time = time.time() - start_test
test_mean_simulation_time = test_evaluation_time/len(benchmark._test_dataset.get_simulations_sizes())

start_test_ood = time.time()
fc_metrics_test_ood = benchmark.evaluate_simulator(dataset="test_ood",augmented_simulator=augmented_simulator,eval_batch_size=256000 )
test_ood_evaluation_time = time.time() - start_test_ood
test_ood_mean_simulation_time = test_ood_evaluation_time/len(benchmark._test_ood_dataset.get_simulations_sizes())

The result returned by this method contains all the metrics.

## Performance of an augmented simulator <a id="bench1-comp"></a>

### Machine learning metrics 

And now we can assess the performance of the "augmented simulator". If we want to retrieve the ML metrics on the test dataset we can use:

In [ ]:
ML_metrics = "ML"
dataset_name = "test"
print("Fully Connected Augmented Simulator")
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("ML metrics", fc_metrics_test[dataset_name][ML_metrics]))

### Physic compliance
A trained augmented simulator could make some errors when verifying physics compliances.

In [ ]:
physic_compliances = "Physics"
dataset_name = "test"
physical_metrics = fc_metrics_test[dataset_name][physic_compliances]
print(physical_metrics)

### OOD
Assess whether the augmented simulator perform well on data that were not seen during training

In [ ]:
ood_ml_metrics = fc_metrics_test_ood["test_ood"][ML_metrics]
ood_physical_metrics = fc_metrics_test_ood["test_ood"][physic_compliances]
print(ood_ml_metrics)
print(ood_physical_metrics)

## Compute the score

In order to focus here on the process rather on the precise details, we skip the explanations regarding the score computation. For more details, we refer to [this notebook](https://github.com/IRT-SystemX/ml4physim_startingkit/blob/main/5_Scoring.ipynb)

We define the thresholds and the airfoil configuration (can not be changed by the user and does not depend on the augmented solver).

In [ ]:
thresholds={"x-velocity":(0.05,0.1,"min"),
            "y-velocity":(0.05,0.1,"min"),
            "pressure":(0.05,0.1,"min"),
            "pressure_surfacic":(0.05,0.1,"min"),
            "turbulent_viscosity":(0.05,0.1,"min"),
            "mean_relative_drag":(0.05,0.1,"min"),
            "mean_relative_lift":(0.05,0.1,"min"),
            "spearman_correlation_drag":(0.5,0.8,"max"),
            "spearman_correlation_lift":(0.5,0.8,"max")           
}

In [ ]:
configuration={
        "coefficients":{"ML":0.4,"OOD":0.3,"Physics":0.3},
        "ratioRelevance":{"Speed-up":0.25,"Accuracy":0.75},
        "valueByColor":{"g":2,"o":1,"r":0},
        "maxSpeedRatioAllowed":10000,
        "reference_mean_simulation_time":1500
    }

We collect the appropriate metrics for each variables from the evaluation results

In [ ]:
ml_metrics = fc_metrics_test["test"][ML_metrics]["MAPE"]
ml_metrics["pressure_surfacic"] = fc_metrics_test["test"][ML_metrics]["MAPE_surfacic"]["pressure"]

phy_variables_to_keep = ["mean_relative_drag","mean_relative_lift","spearman_correlation_drag","spearman_correlation_lift"]
phy_metrics = {phy_variable:fc_metrics_test["test"][physic_compliances][phy_variable] for phy_variable in phy_variables_to_keep}

ml_ood_metrics = fc_metrics_test_ood["test_ood"][ML_metrics]["MAPE"]
ml_ood_metrics["pressure_surfacic"] = fc_metrics_test_ood["test_ood"][ML_metrics]["MAPE_surfacic"]["pressure"]
phy_ood_metrics = {phy_variable:fc_metrics_test_ood["test_ood"][physic_compliances][phy_variable] for phy_variable in phy_variables_to_keep}
ood_metrics = {**ml_ood_metrics,**phy_ood_metrics}

all_metrics={
    ML_metrics:ml_metrics,
    physic_compliances:phy_metrics,
    "OOD":ood_metrics
}

print(all_metrics)

We compute the speed-up for the test dataset and out of distribution dataset

In [ ]:
reference_mean_simulation_time=configuration["reference_mean_simulation_time"]
speedUp={
        ML_metrics:reference_mean_simulation_time/test_mean_simulation_time,
         "OOD":reference_mean_simulation_time/test_mean_simulation_time
        }

We define the function to compute the global score

In [ ]:
import math
def compute_global_score(metrics,speedUp,configuration,thresholds):
    accuracyResults=dict()
    for subcategoryName, subcategoryVal in metrics.items():
        accuracyResults[subcategoryName]=[]
        for variableName, variableError in subcategoryVal.items():
            thresholdMin,thresholdMax,evalType=thresholds[variableName]
            if evalType=="min":
                if variableError<thresholdMin:
                    accuracyEval="g"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="r"
            elif evalType=="max":
                if variableError<thresholdMin:
                    accuracyEval="r"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="g"
    
            accuracyResults[subcategoryName].append(accuracyEval)

    print("accuracyResults: ",accuracyResults)
    
    coefficients=configuration["coefficients"]
    ratioRelevance=configuration["ratioRelevance"]
    valueByColor=configuration["valueByColor"]
    maxSpeedRatioAllowed=configuration["maxSpeedRatioAllowed"]
    mlSubscore=0
    accuracyMaxPoints=ratioRelevance["Accuracy"]
    accuracyResult=sum([valueByColor[color] for color in accuracyResults["ML"]])
    accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["ML"])*max(valueByColor.values()))
    mlSubscore+=accuracyResult
    speedUpMaxPoints=ratioRelevance["Speed-up"]
    speedUpResult=max(0,min(math.log10(speedUp["ML"])/math.log10(maxSpeedRatioAllowed),1))
    speedUpResult=speedUpResult*speedUpMaxPoints
    mlSubscore+=speedUpResult

    accuracyResult=sum([valueByColor[color] for color in accuracyResults["Physics"]])
    accuracyResult=accuracyResult/(len(accuracyResults["Physics"])*max(valueByColor.values()))
    physicsSubscore=accuracyResult

    oodSubscore=0
    accuracyMaxPoints=ratioRelevance["Accuracy"]
    accuracyResult=sum([valueByColor[color] for color in accuracyResults["OOD"]])
    accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["OOD"])*max(valueByColor.values()))
    oodSubscore+=accuracyResult
    
    speedUpMaxPoints=ratioRelevance["Speed-up"]
    speedUpResult=max(0,min(math.log10(speedUp["OOD"])/math.log10(maxSpeedRatioAllowed),1))
    speedUpResult=speedUpResult*speedUpMaxPoints
    oodSubscore+=speedUpResult

    globalScore=100*(coefficients["ML"]*mlSubscore+coefficients["Physics"]*physicsSubscore+coefficients["OOD"]*oodSubscore)
    return globalScore

And finally we compute the global score

In [ ]:
globalScore = compute_global_score(all_metrics,speedUp,configuration,thresholds)
print(globalScore)